In [86]:
import numpy as np
import pandas as pd

In [87]:
shots = pd.read_csv('Kobe/training_data.csv')
shots.head()

,shot_id,game_date,action_type,loc_x,loc_y,total_seconds_remaining,period,playoffs,season,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,opponent,Home_vs_Away,lat,lon
0,2,10/31/2000,Jump Shot,-157,0,622,1,0,2000-01,15,0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,POR,Away,34.0443,-118.4268
1,4,10/31/2000,Jump Shot,138,175,412,1,0,2000-01,22,0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,POR,Away,33.8693,-118.1318
2,6,10/31/2000,Jump Shot,-145,-11,572,3,0,2000-01,14,0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,POR,Away,34.0553,-118.4148
3,10,10/31/2000,Running Jump Shot,-33,125,216,3,0,2000-01,12,0,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,POR,Away,33.9193,-118.3028
4,11,10/31/2000,Jump Shot,-94,238,116,3,0,2000-01,25,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,POR,Away,33.8063,-118.3638


In [88]:
shots.drop("lat",axis=1,inplace=True)
shots.drop("lon",axis=1,inplace=True)
shots.drop("shot_id",axis=1,inplace=True)
shots.drop("game_date",axis=1,inplace=True)
shots.drop("period",axis=1,inplace=True)
shots.drop("season",axis=1,inplace=True)
shots.drop("shot_type",axis=1,inplace=True)
shots.drop("shot_zone_area",axis=1,inplace=True)
shots.drop("shot_zone_basic",axis=1,inplace=True)
shots.drop("shot_zone_range",axis=1,inplace=True)
shots.drop("Home_vs_Away",axis=1,inplace=True)
shots.drop("playoffs",axis=1,inplace=True)
shots.drop("opponent",axis=1,inplace=True)
shots.drop("total_seconds_remaining",axis=1,inplace=True)
shots.head()

,action_type,loc_x,loc_y,shot_distance,shot_made_flag
0,Jump Shot,-157,0,15,0
1,Jump Shot,138,175,22,0
2,Jump Shot,-145,-11,14,0
3,Running Jump Shot,-33,125,12,0
4,Jump Shot,-94,238,25,0


In [89]:
action_columns = pd.get_dummies(shots['action_type'],prefix = "action")
shots = pd.concat([shots, action_columns], axis=1)
shots.drop('action_type', axis=1, inplace=True)

In [91]:
shots.drop(['action_Cutting Layup Shot','action_Driving Floating Bank Jump Shot',
           'action_Hook Bank Shot','action_Putback Slam Dunk Shot','action_Running Finger Roll Shot',
           'action_Running Slam Dunk Shot','action_Tip Layup Shot','action_Turnaround Finger Roll Shot'],axis=1,inplace =True)



In [92]:
shots.head()

,loc_x,loc_y,shot_distance,shot_made_flag,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,action_Driving Finger Roll Shot,...,action_Running Pull-Up Jump Shot,action_Running Reverse Layup Shot,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,-157,0,15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,138,175,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-145,-11,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-33,125,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-94,238,25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
X = shots.drop("shot_made_flag", axis=1)
y = shots["shot_made_flag"]
print(X.shape, y.shape)

(25697, 50) (25697,)


In [94]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


#Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

#Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [95]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=50))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [96]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
 - 2s - loss: 0.6267 - acc: 0.6734
Epoch 2/10
 - 1s - loss: 0.6159 - acc: 0.6780
Epoch 3/10
 - 1s - loss: 0.6127 - acc: 0.6798
Epoch 4/10
 - 1s - loss: 0.6110 - acc: 0.6807
Epoch 5/10
 - 1s - loss: 0.6103 - acc: 0.6801
Epoch 6/10
 - 1s - loss: 0.6099 - acc: 0.6807
Epoch 7/10
 - 1s - loss: 0.6095 - acc: 0.6803
Epoch 8/10
 - 1s - loss: 0.6091 - acc: 0.6808
Epoch 9/10
 - 1s - loss: 0.6091 - acc: 0.6810
Epoch 10/10
 - 1s - loss: 0.6089 - acc: 0.6810


In [97]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.6128368839215675, Accuracy: 0.682645914443272


In [98]:
shots_test = pd.read_csv('Kobe/test_data.csv')
shots_test.head()

,shot_id,game_date,action_type,loc_x,loc_y,total_seconds_remaining,period,playoffs,season,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,opponent,Home_vs_Away,lat,lon
0,1,10/31/2000,Jump Shot,167,72,627,1,0,2000-01,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,POR,Away,33.9723,-118.1028
1,8,10/31/2000,Jump Shot,1,28,485,3,0,2000-01,2,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,POR,Away,34.0163,-118.2688
2,17,11/1/2000,Driving Layup Shot,0,0,1,1,0,2000-01,0,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,UTA,Home,34.0443,-118.2698
3,20,11/1/2000,Driving Layup Shot,0,0,646,3,0,2000-01,0,NaN,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,UTA,Home,34.0443,-118.2698
4,33,11/4/2000,Jump Shot,163,76,686,1,0,2000-01,17,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,VAN,Away,33.9683,-118.1068


In [99]:
shots_test.drop(["lat","lon","shot_id","game_date","period","season","shot_type","shot_zone_area","shot_zone_basic","shot_zone_range","Home_vs_Away","playoffs","opponent","total_seconds_remaining"],axis=1,inplace=True)
shots_test.head()

,action_type,loc_x,loc_y,shot_distance,shot_made_flag
0,Jump Shot,167,72,18,NaN
1,Jump Shot,1,28,2,NaN
2,Driving Layup Shot,0,0,0,NaN
3,Driving Layup Shot,0,0,0,NaN
4,Jump Shot,163,76,17,NaN


In [101]:
act_columns = pd.get_dummies(shots_test['action_type'],prefix = "action")
shots_test = pd.concat([shots_test, act_columns], axis=1)
shots_test.drop('action_type', axis=1, inplace=True)

In [102]:
shots_test.head()

,loc_x,loc_y,shot_distance,shot_made_flag,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Cutting Finger Roll Layup Shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,...,action_Running Reverse Layup Shot,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway Bank Jump Shot,action_Turnaround Fadeaway shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
0,167,72,18,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,28,2,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,163,76,17,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
shots_test.drop('shot_made_flag',axis=1,inplace=True)
shots_test.drop('action_Cutting Finger Roll Layup Shot',axis=1,inplace=True)
shots_test.drop('action_Turnaround Fadeaway Bank Jump Shot',axis=1,inplace=True)



In [104]:
output = model.predict_proba(shots_test).astype(float)
#shots_test['shot_made_flag']=output

In [57]:
shots_test.tail()

,loc_x,loc_y,shot_distance,action_Alley Oop Dunk Shot,action_Alley Oop Layup shot,action_Cutting Layup Shot,action_Driving Bank shot,action_Driving Dunk Shot,action_Driving Finger Roll Layup Shot,action_Driving Finger Roll Shot,...,action_Running Tip Shot,action_Slam Dunk Shot,action_Step Back Jump shot,action_Tip Layup Shot,action_Tip Shot,action_Turnaround Bank shot,action_Turnaround Fadeaway shot,action_Turnaround Finger Roll Shot,action_Turnaround Hook Shot,action_Turnaround Jump Shot
25692,NaN,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25693,NaN,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25694,NaN,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25695,NaN,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25696,NaN,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


array([ 0.96807021,  0.0319298 ])

In [105]:
dummy=pd.read_csv('Kobe/test_data.csv')
submission=pd.DataFrame()
submission['shot_id']=dummy['shot_id']
submission.head()
   

,shot_id
0,1
1,8
2,17
3,20
4,33


In [106]:
submission['shot_made_flag']=0

In [107]:
submission.head()

,shot_id,shot_made_flag
0,1,0
1,8,0
2,17,0
3,20,0
4,33,0


In [114]:
lists = []
for result in output:
    lists.append(result[0])
 
    

In [116]:
submission['shot_made_flag'] = lists

In [118]:
submission.tail()

,shot_id,shot_made_flag
4995,30669,1.000000
4996,30681,0.546913
4997,30683,1.000000
4998,30687,1.000000
4999,30694,0.546913


In [119]:
submission.to_csv('neural_net_test.csv',index=False)

In [ ]:
# output = model.predict(test).astype(int)
# df_output = pd.DataFrame()
# file = os.path.join('Kobe', 'test_data.csv')
# aux = pd.read_csv(file)
# df_output['PassengerId'] = aux['PassengerId']
# df_output['Survived'] = output
# df_output[['PassengerId','Survived']].to_csv('gridsearch_random_8.csv', index=False)